# Portfolio Analytics AI - Getting Started

This notebook demonstrates the basic functionality of the Portfolio Analytics AI package.

## Overview
- Create a portfolio with multiple assets
- Load historical data
- Calculate basic performance metrics
- Optimize portfolio weights
- Visualize results

In [ ]:
# Import necessary libraries
import sys
import os

# Add src to path for imports
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from portfolio_analytics import Portfolio, DataProvider
from portfolio_analytics.visualization import PortfolioVisualizer

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Create a Portfolio

Let's create a portfolio with some popular tech stocks.

In [ ]:
# Define portfolio symbols
symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']

# Create portfolio with equal weights
portfolio = Portfolio(symbols, name="Tech Portfolio")

print(f"Created portfolio: {portfolio}")
print(f"Initial weights: {dict(zip(symbols, portfolio.weights))}")

## 2. Load Historical Data

We'll use the DataProvider to fetch historical price data from Yahoo Finance.

In [ ]:
# Initialize data provider
data_provider = DataProvider(source="yahoo")

# Load historical data (last 3 years)
try:
    portfolio.load_data(
        data_provider=data_provider,
        start_date="2021-01-01",
        end_date="2024-01-01"
    )
    print("Data loaded successfully!")
    print(f"Data shape: {portfolio.data.shape}")
    print(f"Date range: {portfolio.data.index[0]} to {portfolio.data.index[-1]}")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Note: This example requires internet connection and yfinance package")

## 3. Basic Portfolio Analysis

In [ ]:
# Calculate basic metrics
if portfolio.data is not None:
    print("Portfolio Performance Metrics:")
    print(f"Annual Return: {portfolio.annual_return():.2%}")
    print(f"Annual Volatility: {portfolio.annual_volatility():.2%}")
    print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.3f}")
    print(f"Value at Risk (95%): {portfolio.calculate_var(0.95):.2%}")
    
    # Get comprehensive performance metrics
    performance_metrics = portfolio.get_performance_metrics()
    print("\nDetailed Performance Metrics:")
    for metric, value in performance_metrics.items():
        if isinstance(value, float):
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")
else:
    print("No data available for analysis. Please check data loading step.")

## 4. Portfolio Optimization

Let's optimize the portfolio to maximize the Sharpe ratio.

In [ ]:
if portfolio.data is not None:
    try:
        # Store original weights
        original_weights = portfolio.weights.copy()
        
        # Optimize for maximum Sharpe ratio
        optimized_weights = portfolio.optimize(method="max_sharpe")
        
        print("Portfolio Optimization Results:")
        print("\nOriginal vs Optimized Weights:")
        
        comparison_df = pd.DataFrame({
            'Symbol': symbols,
            'Original': original_weights,
            'Optimized': optimized_weights,
            'Change': optimized_weights - original_weights
        })
        
        print(comparison_df.round(4))
        
        # Calculate metrics for optimized portfolio
        print(f"\nOptimized Portfolio Metrics:")
        print(f"Annual Return: {portfolio.annual_return():.2%}")
        print(f"Annual Volatility: {portfolio.annual_volatility():.2%}")
        print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.3f}")
        
    except Exception as e:
        print(f"Optimization failed: {e}")
        print("Note: Optimization requires scipy package")
else:
    print("No data available for optimization.")

## 5. Visualization

Let's create some visualizations of our portfolio.

In [ ]:
if portfolio.data is not None:
    # Initialize visualizer
    visualizer = PortfolioVisualizer()
    
    # Plot price history
    try:
        visualizer.plot_price_history(
            portfolio.data,
            title="Portfolio Assets Price History",
            figsize=(12, 6)
        )
    except Exception as e:
        print(f"Error plotting price history: {e}")
    
    # Plot correlation matrix
    try:
        visualizer.plot_correlation_matrix(
            portfolio.returns,
            title="Asset Correlation Matrix",
            figsize=(8, 6)
        )
    except Exception as e:
        print(f"Error plotting correlation matrix: {e}")
    
    # Plot portfolio composition
    try:
        weights_dict = dict(zip(symbols, portfolio.weights))
        visualizer.plot_portfolio_composition(
            weights_dict,
            title="Optimized Portfolio Composition",
            chart_type="pie",
            figsize=(8, 8)
        )
    except Exception as e:
        print(f"Error plotting portfolio composition: {e}")
        
    # Plot cumulative returns
    try:
        portfolio_returns = (portfolio.returns * portfolio.weights).sum(axis=1)
        visualizer.plot_cumulative_returns(
            portfolio_returns,
            title="Portfolio Cumulative Returns",
            figsize=(12, 6)
        )
    except Exception as e:
        print(f"Error plotting cumulative returns: {e}")
else:
    print("No data available for visualization.")

## 6. Risk Analysis

In [ ]:
if portfolio.data is not None:
    # Calculate comprehensive risk metrics
    portfolio_returns = (portfolio.returns * portfolio.weights).sum(axis=1)
    
    risk_metrics = portfolio.risk_model.calculate_risk_metrics(
        portfolio_returns,
        confidence_levels=[0.95, 0.99]
    )
    
    print("Risk Analysis:")
    print("=" * 40)
    
    for metric, value in risk_metrics.items():
        if isinstance(value, float):
            if 'var' in metric.lower() or 'expected_shortfall' in metric.lower():
                print(f"{metric}: {value:.2%}")
            elif 'ratio' in metric.lower():
                print(f"{metric}: {value:.3f}")
            else:
                print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")
else:
    print("No data available for risk analysis.")

## 7. Portfolio Summary

In [ ]:
# Generate comprehensive portfolio summary
summary = portfolio.summary()

print("Portfolio Summary:")
print("=" * 50)

for key, value in summary.items():
    if isinstance(value, dict):
        print(f"\n{key.upper()}:")
        for subkey, subvalue in value.items():
            print(f"  {subkey}: {subvalue}")
    elif isinstance(value, list):
        print(f"{key}: {', '.join(map(str, value))}")
    elif isinstance(value, float):
        if any(metric in key.lower() for metric in ['return', 'volatility', 'var']):
            print(f"{key}: {value:.2%}")
        else:
            print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

## Conclusion

This notebook demonstrated the basic functionality of the Portfolio Analytics AI package:

1. **Portfolio Creation**: Created a diversified tech portfolio
2. **Data Loading**: Fetched historical price data from Yahoo Finance
3. **Performance Analysis**: Calculated key performance metrics
4. **Optimization**: Optimized weights to maximize Sharpe ratio
5. **Visualization**: Created charts to visualize portfolio performance
6. **Risk Analysis**: Analyzed various risk metrics including VaR

### Next Steps

- Explore other optimization methods (minimum variance, target return)
- Add benchmark comparison (e.g., S&P 500)
- Implement more advanced risk models
- Create custom constraints for optimization
- Build interactive dashboards